In [3]:
### import software packages that we'll use
import os
import pickle
import numpy as np
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
from torchsummary import summary
import wandb

In [4]:
#get data
data_path = 'dataset.pkl'
with open(data_path, 'rb') as handle:
    data = pickle.load(handle)

#remove unecessary column
data.drop(labels='cell',axis=1,inplace=True)
data = data.to_numpy()

In [31]:
#define model class
"""
class Model(nn.Module):
     
    def __init__(self, input_size):
        super().__init__()
        
        #fully connected layers:
        self.l1 = nn.Linear(input_size, 16)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.sigmoid(x)
        return x
"""

In [48]:
#define Pytorch Lightning module (executes during training)
class Classifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.l1 = nn.Linear(64, 16)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.sigmoid(x)
        return x
    
    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = self(x)
        loss = F.binary_cross_entropy(x, y.reshape(-1,1).float())
        self.log("train_loss",loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, y = batch
        x = self(x)
        loss = F.binary_cross_entropy(x, y.reshape(-1,1).float())
        self.log("val_loss",loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [49]:
#define Pytorch mappable dataset
class PBMCDataset(Dataset):
    def __init__(self, data):
        self.embeddings = data[:,0]
        self.labels = data[:,1]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        embedding = self.embeddings[idx]
        label = self.labels[idx]
        
        return torch.tensor(embedding), torch.tensor(label)

In [50]:
#training and validation data split
train_set, val_set = torch.utils.data.random_split(PBMCDataset(data), [2000,834])

In [51]:
#initialize weights and biases logger
wandb_logger = WandbLogger(project='classifier')

In [52]:
#define dataloaders
train_loader = DataLoader(train_set)
valid_loader = DataLoader(val_set)

#instantiate pl.LightningModule
classifier = Classifier()

#train model
trainer = Trainer(logger=wandb_logger)
trainer.fit(classifier, train_loader, valid_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [4]

  | Name    | Type    | Params
------------------------------------
0 | l1      | Linear  | 1.0 K 
1 | relu    | ReLU    | 0     
2 | l2      | Linear  | 17    
3 | sigmoid | Sigmoid | 0     
------------------------------------
1.1 K     Trainable params
0         Non-trainable params
1.1 K     Total params
0.004     Total estimated model params size (MB)


Epoch 0: 100%|█| 2000/2000 [00:06<00:00, 330.96it/s, v_num=h5et
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|         | 0/834 [00:00<?, ?it/s]
Validation DataLoader 0:   0%| | 1/834 [00:00<00:02, 307.23it/s
Validation DataLoader 0:   0%| | 2/834 [00:00<00:02, 390.19it/s
Validation DataLoader 0:   0%| | 3/834 [00:00<00:01, 422.61it/s
Validation DataLoader 0:   0%| | 4/834 [00:00<00:02, 365.55it/s
Validation DataLoader 0:   1%| | 5/834 [00:00<00:02, 338.43it/s
Validation DataLoader 0:   1%| | 6/834 [00:00<00:02, 328.15it/s
Validation DataLoader 0:   1%| | 7/834 [00:00<00:02, 327.36it/s
Validation DataLoader 0:   1%| | 8/834 [00:00<00:02, 325.63it/s
Validation DataLoader 0:   1%| | 9/834 [00:00<00:02, 326.95it/s
Validation DataLoader 0:   1%| | 10/834 [00:00<00:02, 326.63it/
Validation DataLoader 0:   1%| | 11/834 [00:00<00:02, 326.36it/
Validation DataLoader 0:   1%| | 12/834 [00:00<00:02, 325.78it/
Validation DataLoader 0:   2%| | 13/834 [00:00<00:02, 325.53it/
Validatio

Validation DataLoader 0:  29%|▎| 241/834 [00:00<00:01, 356.22it
Validation DataLoader 0:  29%|▎| 242/834 [00:00<00:01, 356.09it
Validation DataLoader 0:  29%|▎| 243/834 [00:00<00:01, 355.95it
Validation DataLoader 0:  29%|▎| 244/834 [00:00<00:01, 355.79it
Validation DataLoader 0:  29%|▎| 245/834 [00:00<00:01, 355.66it
Validation DataLoader 0:  29%|▎| 246/834 [00:00<00:01, 355.52it
Validation DataLoader 0:  30%|▎| 247/834 [00:00<00:01, 355.38it
Validation DataLoader 0:  30%|▎| 248/834 [00:00<00:01, 355.24it
Validation DataLoader 0:  30%|▎| 249/834 [00:00<00:01, 355.10it
Validation DataLoader 0:  30%|▎| 250/834 [00:00<00:01, 354.96it
Validation DataLoader 0:  30%|▎| 251/834 [00:00<00:01, 354.83it
Validation DataLoader 0:  30%|▎| 252/834 [00:00<00:01, 354.66it
Validation DataLoader 0:  30%|▎| 253/834 [00:00<00:01, 354.55it
Validation DataLoader 0:  30%|▎| 254/834 [00:00<00:01, 354.39it
Validation DataLoader 0:  31%|▎| 255/834 [00:00<00:01, 354.26it
Validation DataLoader 0:  31%|▎| 256/834

Validation DataLoader 0:  58%|▌| 485/834 [00:01<00:01, 346.91it
Validation DataLoader 0:  58%|▌| 486/834 [00:01<00:01, 347.06it
Validation DataLoader 0:  58%|▌| 487/834 [00:01<00:00, 347.17it
Validation DataLoader 0:  59%|▌| 488/834 [00:01<00:00, 347.36it
Validation DataLoader 0:  59%|▌| 489/834 [00:01<00:00, 347.56it
Validation DataLoader 0:  59%|▌| 490/834 [00:01<00:00, 347.78it
Validation DataLoader 0:  59%|▌| 491/834 [00:01<00:00, 347.84it
Validation DataLoader 0:  59%|▌| 492/834 [00:01<00:00, 348.03it
Validation DataLoader 0:  59%|▌| 493/834 [00:01<00:00, 348.22it
Validation DataLoader 0:  59%|▌| 494/834 [00:01<00:00, 348.43it
Validation DataLoader 0:  59%|▌| 495/834 [00:01<00:00, 348.62it
Validation DataLoader 0:  59%|▌| 496/834 [00:01<00:00, 348.81it
Validation DataLoader 0:  60%|▌| 497/834 [00:01<00:00, 349.01it
Validation DataLoader 0:  60%|▌| 498/834 [00:01<00:00, 349.07it
Validation DataLoader 0:  60%|▌| 499/834 [00:01<00:00, 349.28it
Validation DataLoader 0:  60%|▌| 500/834

Validation DataLoader 0:  87%|▊| 729/834 [00:02<00:00, 352.04it
Validation DataLoader 0:  88%|▉| 730/834 [00:02<00:00, 351.98it
Validation DataLoader 0:  88%|▉| 731/834 [00:02<00:00, 351.93it
Validation DataLoader 0:  88%|▉| 732/834 [00:02<00:00, 351.87it
Validation DataLoader 0:  88%|▉| 733/834 [00:02<00:00, 351.81it
Validation DataLoader 0:  88%|▉| 734/834 [00:02<00:00, 351.75it
Validation DataLoader 0:  88%|▉| 735/834 [00:02<00:00, 351.69it
Validation DataLoader 0:  88%|▉| 736/834 [00:02<00:00, 351.66it
Validation DataLoader 0:  88%|▉| 737/834 [00:02<00:00, 351.59it
Validation DataLoader 0:  88%|▉| 738/834 [00:02<00:00, 351.54it
Validation DataLoader 0:  89%|▉| 739/834 [00:02<00:00, 351.49it
Validation DataLoader 0:  89%|▉| 740/834 [00:02<00:00, 351.46it
Validation DataLoader 0:  89%|▉| 741/834 [00:02<00:00, 351.40it
Validation DataLoader 0:  89%|▉| 742/834 [00:02<00:00, 351.35it
Validation DataLoader 0:  89%|▉| 743/834 [00:02<00:00, 351.29it
Validation DataLoader 0:  89%|▉| 744/834

...

Validation DataLoader 0:  53%|▌| 443/834 [00:01<00:00, 418.48it
Validation DataLoader 0:  53%|▌| 444/834 [00:01<00:00, 418.65it
Validation DataLoader 0:  53%|▌| 445/834 [00:01<00:00, 418.85it
Validation DataLoader 0:  53%|▌| 446/834 [00:01<00:00, 419.03it
Validation DataLoader 0:  54%|▌| 447/834 [00:01<00:00, 419.23it
Validation DataLoader 0:  54%|▌| 448/834 [00:01<00:00, 419.43it
Validation DataLoader 0:  54%|▌| 449/834 [00:01<00:00, 419.66it
Validation DataLoader 0:  54%|▌| 450/834 [00:01<00:00, 419.82it
Validation DataLoader 0:  54%|▌| 451/834 [00:01<00:00, 419.98it
Validation DataLoader 0:  54%|▌| 452/834 [00:01<00:00, 420.17it
Validation DataLoader 0:  54%|▌| 453/834 [00:01<00:00, 420.37it
Validation DataLoader 0:  54%|▌| 454/834 [00:01<00:00, 420.56it
Validation DataLoader 0:  55%|▌| 455/834 [00:01<00:00, 420.60it
Validation DataLoader 0:  55%|▌| 456/834 [00:01<00:00, 420.77it
Validation DataLoader 0:  55%|▌| 457/834 [00:01<00:00, 420.96it
Validation DataLoader 0:  55%|▌| 458/834

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 6: 100%|█| 2000/2000 [00:05<00:00, 337.47it/s, v_num=h5et
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|         | 0/834 [00:00<?, ?it/s]
Validation DataLoader 0:   0%| | 1/834 [00:00<00:02, 353.50it/s
Validation DataLoader 0:   0%| | 2/834 [00:00<00:02, 399.76it/s
Validation DataLoader 0:   0%| | 3/834 [00:00<00:01, 439.26it/s
Validation DataLoader 0:   0%| | 4/834 [00:00<00:01, 457.88it/s
Validation DataLoader 0:   1%| | 5/834 [00:00<00:01, 472.85it/s
Validation DataLoader 0:   1%| | 6/834 [00:00<00:01, 481.11it/s
Validation DataLoader 0:   1%| | 7/834 [00:00<00:01, 486.14it/s
Validation DataLoader 0:   1%| | 8/834 [00:00<00:01, 492.71it/s
Validation DataLoader 0:   1%| | 9/834 [00:00<00:01, 496.30it/s
Validation DataLoader 0:   1%| | 10/834 [00:00<00:01, 499.09it/
Validation DataLoader 0:   1%| | 11/834 [00:00<00:01, 502.04it/
Validation DataLoader 0:   1%| | 12/834 [00:00<00:01, 501.78it/
Validation DataLoader 0:   2%| | 13/834 [00:00<00:01, 503.91it/
Validatio

Validation DataLoader 0:  29%|▎| 241/834 [00:00<00:01, 484.18it
Validation DataLoader 0:  29%|▎| 242/834 [00:00<00:01, 483.06it
Validation DataLoader 0:  29%|▎| 243/834 [00:00<00:01, 482.05it
Validation DataLoader 0:  29%|▎| 244/834 [00:00<00:01, 481.00it
Validation DataLoader 0:  29%|▎| 245/834 [00:00<00:01, 479.96it
Validation DataLoader 0:  29%|▎| 246/834 [00:00<00:01, 478.92it
Validation DataLoader 0:  30%|▎| 247/834 [00:00<00:01, 477.97it
Validation DataLoader 0:  30%|▎| 248/834 [00:00<00:01, 476.97it
Validation DataLoader 0:  30%|▎| 249/834 [00:00<00:01, 475.98it
Validation DataLoader 0:  30%|▎| 250/834 [00:00<00:01, 475.04it
Validation DataLoader 0:  30%|▎| 251/834 [00:00<00:01, 474.05it
Validation DataLoader 0:  30%|▎| 252/834 [00:00<00:01, 473.15it
Validation DataLoader 0:  30%|▎| 253/834 [00:00<00:01, 472.17it
Validation DataLoader 0:  30%|▎| 254/834 [00:00<00:01, 470.69it
Validation DataLoader 0:  31%|▎| 255/834 [00:00<00:01, 469.36it
Validation DataLoader 0:  31%|▎| 256/834

Validation DataLoader 0:  58%|▌| 485/834 [00:01<00:00, 390.83it
Validation DataLoader 0:  58%|▌| 486/834 [00:01<00:00, 390.66it
Validation DataLoader 0:  58%|▌| 487/834 [00:01<00:00, 390.48it
Validation DataLoader 0:  59%|▌| 488/834 [00:01<00:00, 390.31it
Validation DataLoader 0:  59%|▌| 489/834 [00:01<00:00, 390.16it
Validation DataLoader 0:  59%|▌| 490/834 [00:01<00:00, 389.98it
Validation DataLoader 0:  59%|▌| 491/834 [00:01<00:00, 389.81it
Validation DataLoader 0:  59%|▌| 492/834 [00:01<00:00, 389.66it
Validation DataLoader 0:  59%|▌| 493/834 [00:01<00:00, 389.49it
Validation DataLoader 0:  59%|▌| 494/834 [00:01<00:00, 389.31it
Validation DataLoader 0:  59%|▌| 495/834 [00:01<00:00, 389.15it
Validation DataLoader 0:  59%|▌| 496/834 [00:01<00:00, 388.98it
Validation DataLoader 0:  60%|▌| 497/834 [00:01<00:00, 388.82it
Validation DataLoader 0:  60%|▌| 498/834 [00:01<00:00, 388.66it
Validation DataLoader 0:  60%|▌| 499/834 [00:01<00:00, 388.51it
Validation DataLoader 0:  60%|▌| 500/834

In [72]:
#after training, predict with the model
model = Classifier()
#load trained model from checkpoint
model = model.load_from_checkpoint("classifier/ihz4h5et/checkpoints/epoch=5-step=12000.ckpt").cuda()
model.eval()

y_hat = []
for d in range(len(data[:,0])):
    with torch.no_grad():
        pred = model(torch.tensor(data[d,0]).cuda())
        y_hat.append(pred.cpu().numpy()[0])

In [75]:
y_hat

[0.52879834,
 0.6483393,
 0.45728728,
 0.4032106,
 0.0016445473,
 0.042009585,
 0.6578892,
 0.790471,
 0.075764984,
 0.012288238,
 0.0041661374,
 0.35567552,
 0.0039904416,
 0.7432866,
 3.705965e-05,
 0.0019764774,
 0.59204656,
 0.7826386,
 0.3586085,
 0.09210616,
 0.0060447655,
 0.5428829,
 0.39771703,
 0.2300884,
 0.7108333,
 2.9001371e-06,
 0.007382021,
 0.0070047304,
 0.77353716,
 0.0072345617,
 0.43571988,
 0.0042139087,
 0.51941496,
 0.7713114,
 0.8272231,
 0.065245226,
 0.03863523,
 0.006525429,
 0.073173486,
 0.0033161067,
 0.6728593,
 0.41580087,
 0.01721523,
 0.7994674,
 0.002706018,
 0.0009948802,
 0.08782581,
 0.8460949,
 0.5053517,
 2.4307296e-06,
 0.3967664,
 0.011632122,
 0.0688197,
 0.00832421,
 0.0011835509,
 0.732456,
 0.055329926,
 0.6027824,
 0.3395144,
 0.5788254,
 0.040473726,
 1.9964402e-05,
 0.6885024,
 0.7944049,
 0.06885471,
 0.35941705,
 0.8016028,
 0.004677476,
 0.047347393,
 0.00015167957,
 0.74756205,
 0.7749744,
 0.1704944,
 0.003018222,
 0.011591476,
 0.